1. Fit KNN and Predict with Train Validate Data Split 
2. Try different k's and parameters


In [1]:
import re
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
data_path_raw = Path.cwd().parent.parent / "data" / "raw"

data_path_preprocessed = Path.cwd().parent.parent / "data" / "processed"


In [3]:
raw_df_processed = pd.read_csv(data_path_preprocessed / "training_set_features_encoded_imputed_standardized.csv", index_col="respondent_id")
labels_df = pd.read_csv(data_path_raw / "training_set_labels.csv", index_col="respondent_id")
test_df_processed = pd.read_csv(data_path_preprocessed / "test_set_features_encoded_imputed_standardized.csv", index_col="respondent_id")

all_raw_df = raw_df_processed.join(labels_df)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    raw_df_processed,
    labels_df,
    shuffle = True,
    test_size = 0.25,
    random_state = 10)

In [14]:
%%time
from sklearn import svm
from sklearn.multioutput import MultiOutputClassifier

svc = MultiOutputClassifier(svm.SVC(probability = True))



svc.fit(X_train, y_train)
#test_predictions = svc.predict(X_test)
#test_probability = svc.predict_proba(X_test)




CPU times: user 6min 51s, sys: 3.41 s, total: 6min 55s
Wall time: 6min 56s


MultiOutputClassifier(estimator=SVC(probability=True))

In [15]:
test_predictions = svc.predict(X_test)

In [16]:
test_probability = svc.predict_proba(X_test)

In [17]:
y_preds = pd.DataFrame(
    {
        "h1n1_vaccine": test_probability[0][:, 1],
        "seasonal_vaccine": test_probability[1][:, 1],
    },
    index = y_test.index
)
print("y_preds.shape:", y_preds.shape)
y_preds.head()

y_preds.shape: (6677, 2)


,h1n1_vaccine,seasonal_vaccine
respondent_id,,
2653,0.075358,0.938279
9506,0.112992,0.483800
23107,0.024983,0.360905
22648,0.045857,0.695473
25589,0.173791,0.780729


In [18]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_preds))

0.8433007377608186


In [13]:
ROC_svc =  [0.8433007377608186 , 411]





ROC_Overview = pd.DataFrame( [ROC_3nn, ROC_5nn, ROC_7nn, ROC_5nn_weighed,ROC_7nn_weighed])
ROC_Overview.columns = ['ROC', 'Time']
ROC_Overview.index = ['ROC_3nn', 'ROC_5nn', 'ROC_7nn', 'ROC_5nn_weighed', 'ROC_7nn_weighed']
ROC_Overview

,ROC,Time
ROC_3nn,0.699621,29.148728
ROC_5nn,0.722953,29.378359
ROC_7nn,0.739735,29.483131
ROC_5nn_weighed,0.726559,33.254742
ROC_7nn_weighed,0.743137,34.526619


In [13]:
from sklearn.metrics import classification_report
#print(classification_report(y_train, train_predictions))
print(classification_report(y_test, test_predictions))

              precision    recall  f1-score   support

           0       0.69      0.42      0.53      1410
           1       0.76      0.74      0.75      3137

   micro avg       0.75      0.64      0.69      4547
   macro avg       0.73      0.58      0.64      4547
weighted avg       0.74      0.64      0.68      4547
 samples avg       0.34      0.32      0.33      4547



/Users/johannes/opt/anaconda3/envs/py37-ds/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johannes/opt/anaconda3/envs/py37-ds/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
9